In [55]:
import pandas as pd

# Load the CSV files into DataFrames
nba_17_18 = pd.read_csv("nba17-18.csv")
nba_18_19 = pd.read_csv("nba18-19.csv")
nba_19_20 = pd.read_csv("nba19-20.csv")
nba_20_21 = pd.read_csv("nba20-21.csv")
nba_21_22 = pd.read_csv("nba21-22.csv")
nba_22_23 = pd.read_csv("nba22-23.csv")
nba_23_24 = pd.read_csv("nba23-24.csv")

def clean_data(df):
    # Drop the first, third, and last two columns
    df = df.drop(df.columns[[0, 2, -1, -2]], axis=1)
    return df

# Clean each dataset
nba_17_18 = clean_data(nba_17_18)
nba_18_19 = clean_data(nba_18_19)
nba_19_20 = clean_data(nba_19_20)
nba_20_21 = clean_data(nba_20_21)
nba_21_22 = clean_data(nba_21_22)
nba_22_23 = clean_data(nba_22_23)
nba_23_24 = clean_data(nba_23_24)

# Function to filter rows where "Team" is '2TM' for each player
def filter_tm(df):
    # For each player, keep the row where "Team" is '2TM' or '3TM' if available, else keep the first row
    def get_tm_or_first(group):
        if "3TM" in group["Team"].values:
            return group[group["Team"] == "3TM"]
        elif "2TM" in group["Team"].values:
            return group[group["Team"] == "2TM"]
        else:
            return group.head(1)
    
    return df.groupby("Player", group_keys=False).apply(get_tm_or_first)

# Apply the filtering function to each dataset
nba_17_18 = filter_tm(nba_17_18)
nba_18_19 = filter_tm(nba_18_19)
nba_19_20 = filter_tm(nba_19_20)
nba_20_21 = filter_tm(nba_20_21)
nba_21_22 = filter_tm(nba_21_22)
nba_22_23 = filter_tm(nba_22_23)
nba_23_24 = filter_tm(nba_23_24)

# Add season identifiers to the column names (excluding the first column)
def add_season_prefix(df, season):
    df.columns = [df.columns[0]] + [f"{season}.{col}" for col in df.columns[1:]]
    return df

nba_17_18 = add_season_prefix(nba_17_18, "17-18")
nba_18_19 = add_season_prefix(nba_18_19, "18-19")
nba_19_20 = add_season_prefix(nba_19_20, "19-20")
nba_20_21 = add_season_prefix(nba_20_21, "20-21")
nba_21_22 = add_season_prefix(nba_21_22, "21-22")
nba_22_23 = add_season_prefix(nba_22_23, "22-23")
nba_23_24 = add_season_prefix(nba_23_24, "23-24")


C:\Users\aasia\AppData\Local\Temp\ipykernel_23824\4277233094.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Player", group_keys=False).apply(get_tm_or_first)
C:\Users\aasia\AppData\Local\Temp\ipykernel_23824\4277233094.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Player", group_keys=False).apply(get_tm_or_first)
C:\Users\aasia\AppData\Local\Temp\ipykernel_

In [53]:
nba_master_data = nba_17_18
for df in [nba_18_19, nba_19_20, nba_20_21, nba_21_22, nba_22_23, nba_23_24]:
    nba_master_data = pd.merge(nba_master_data, df, on="Player", how="outer")